In [ ]:
import json
import re
import boto3
from pprint import pprint as pp
import pandas as pd
import csv
from csv import writer as wt
import os.path
from glob import glob
from io import StringIO

s3_client = boto3.client('s3')

bucket_list = s3_client.list_buckets()
bucket_name = 'data-28-final-project-files-group2'

#=====================================================================

def get_course_date(i):
    filename=i['Key']
    name=filename.split('/')[1].split('_')
    course='_'.join([name[0],name[1]])
    date = name[2].split('.')[0]
#     print(course)
    return course,date


def get_csvs(client,bucket_name,pref):
    """ Takes in a client and bucket name concatenate all the csvs inside
    and return a pandas dataframe
    to update :: with an extra column specifying the course title and class
    name tutor score score ..... course date
    """
    bucket_contents = client.list_objects(Bucket=bucket_name,Prefix=pref)["Contents"]

    academic = pd.DataFrame()

    for i in bucket_contents:
        course_date =get_course_date(i)
        latest_course = client.get_object(Bucket=bucket_name, Key=i["Key"])
        a=pd.read_csv(latest_course["Body"])
        a['course']=course_date[0]
     
        a['start_date']=course_date[1]
        
        academic = pd.concat([academic,a ])

    return academic


def reind(df):
    col = list(df.columns)
    col.remove('course')
    col.remove('start_date')
    col.extend(['course','start_date'])
    df = df.reindex(columns=col)
    df['old index']=df.index
    df = df.reset_index(drop=True)
    return df

def get_csvs_all_courses(client, bucket_name,one_file=True):
    """ Takes in a client and bucket name concatenate all the csvs inside
        and return a pandas dataframe
        to update :: with an extra column specifying the course title and class
        name tutor score score ..... course date
        """
    
    prefixes = ['Academy/Business', 'Academy/Data', 'Academy/Engineering']
    full_list = pd.DataFrame()
    files=[]
    for pref in prefixes:
        b = get_csvs(client, bucket_name, pref)
        b =reind(b)
       
        if one_file:
            full_list = pd.concat([full_list, b])
        else:
    
            files.append(b)
        print(pref,'is done')
    print('done')
    return full_list if one_file else files

folder= "Allcsv"
def upload(client, bucket_name,one_file=True,folder="Allcsv"):
    
    subjects =  (get_csvs_all_courses(s3_client,bucket_name,one_file=False))
    filenames=['Business.csv','data.csv','Engineering.csv']
#   bucket_name,filename,foolder,dum
    for i in range(len(filenames)):
        key =f"{folder}/{filenames[i]}"
        
        print('uploading ',filenames[i])
       
        content=  subjects[i].to_csv()
#         subjects[i].to_csv(filenames[i])
        s3_client.put_object(Bucket=bucket_name,Body=subjects[i].to_csv(), Key=key)
    return 'done'